In [15]:
import matplotlib.pyplot as plt

In [30]:
from posteriordb import PosteriorDatabase

pdb_path = "/workspaces/jupyter-data/posteriordb/posterior_database/"
my_pdb = PosteriorDatabase(pdb_path)
model_name = "eight_schools"
model = my_pdb.model(f"{model_name}_centered")
data = my_pdb.data(model_name)

In [31]:
print(model.stan_code())

data {
  int <lower=0> J; // number of schools
  real y[J]; // estimated treatment
  real<lower=0> sigma[J]; // std of estimated effect
}
parameters {
  real theta[J]; // treatment effect in school j
  real mu; // hyper-parameter of mean
  real<lower=0> tau; // hyper-parameter of sdv
}
model {
  tau ~ cauchy(0, 5); // a non-informative prior
  theta ~ normal(mu, tau);
  y ~ normal(theta, sigma);
  mu ~ normal(0, 5);
}



In [32]:
import torch

sigma = torch.tensor(data.values()['sigma'])
y = torch.tensor(data.values()['y'])

In [5]:
import torch
import torch.distributions as dist
import flowtorch.bijectors
import flowtorch.params

flow = flowtorch.bijectors.AffineAutoregressive(
    flowtorch.params.DenseAutoregressive(
        hidden_dims=(32,32,32)
    )
)
base_dist = dist.Normal(torch.zeros(8), torch.ones(8))
new_dist, params = flow(base_dist)

In [6]:
def p(samples):
    return (
        dist.Independent(dist.Normal(samples, sigma), 1).log_prob(y) +
        dist.Normal(dist.Normal(0, 5).sample(), dist.HalfCauchy(5).sample()).log_prob(samples).sum(1)
    )

In [14]:
import torch.optim as optim
import torch.nn as nn
from tqdm.auto import tqdm

data = []

for trial in range(100):
    for num_layers in range(1,5):
        flow = flowtorch.bijectors.AffineAutoregressive(
            flowtorch.params.DenseAutoregressive(
                hidden_dims=(32,)*num_layers
            )
        )
        base_dist = dist.Normal(torch.zeros(8), torch.ones(8))
        new_dist, params = flow(base_dist)
        optimizer = optim.Adam(params.parameters(), lr=1e-3)

        n = 1000
        for _ in tqdm(range(int(1e3))):
            optimizer.zero_grad()
            samples = new_dist.rsample((n,))
            #samples = target.sample((1000,))
            log_q = new_dist.log_prob(samples)
            log_p = p(samples)
            loss = (log_q - log_p).mean()
            #loss = -log_q.mean()
            loss.backward(retain_graph=True)
            optimizer.step()


            z = new_dist.sample((n,))
            marginal_lik = torch.logsumexp(p(z) - new_dist.log_prob(z) - torch.log(torch.tensor([n])), dim=0)
            tqdm.write(f"ELBO: {-loss}, log p(y): {marginal_lik}", end='')

        data.append({
            'method': 'ADVI',
            'num_layers': num_layers,
            'elbo': -loss.item(),
            'log_p_y': marginal_lik.item(),
        })

        # TAF, train just DF
        dfs = nn.Parameter(data=torch.ones(1)*10)
        new_dist.base_dist = dist.StudentT(df=dfs*torch.ones(8), loc=torch.zeros(8), scale=torch.ones(8))
        optimizer = optim.Adam([dfs], lr=1e-2)
        for _ in tqdm(range(int(1e3))):
            optimizer.zero_grad()
            samples = new_dist.rsample((n,))
            log_q = new_dist.log_prob(samples)
            log_p = p(samples)
            loss = (log_q - log_p).mean()
            loss.backward(retain_graph=True)
            optimizer.step()
            new_dist.base_dist = dist.StudentT(df=dfs*torch.ones(8), loc=torch.zeros(8), scale=torch.ones(8))


            z = new_dist.sample((n,))
            marginal_lik = torch.logsumexp(p(z) - new_dist.log_prob(z) - torch.log(torch.tensor([n])), dim=0)
            tqdm.write(f"ELBO: {-loss}, log p(y): {marginal_lik}, dfs: {dfs[0:2]}", end='')
            
        data.append({
            'method': 'TAF',
            'num_layers': num_layers,
            'elbo': -loss.item(),
            'log_p_y': marginal_lik.item(),
        })

        # TAF, train just DF
        dfs = nn.Parameter(data=dfs.data*torch.ones(8))
        new_dist.base_dist = dist.StudentT(df=dfs, loc=torch.zeros(8), scale=torch.ones(8))
        optimizer = optim.Adam([dfs], lr=1e-2)
        for _ in tqdm(range(int(1e3))):
            optimizer.zero_grad()
            samples = new_dist.rsample((n,))
            log_q = new_dist.log_prob(samples)
            log_p = p(samples)
            loss = (log_q - log_p).mean()
            loss.backward(retain_graph=True)
            optimizer.step()


            z = new_dist.sample((n,))
            marginal_lik = torch.logsumexp(p(z) - new_dist.log_prob(z) - torch.log(torch.tensor([n])), dim=0)
            tqdm.write(f"ELBO: {-loss}, log p(y): {marginal_lik}, dfs: {dfs[0:2]}", end='')
            
        data.append({
            'method': 'ATAF',
            'num_layers': num_layers,
            'elbo': -loss.item(),
            'log_p_y': marginal_lik.item(),
        })

ELBO: -57.37480545043945, log p(y): -41.7869720458984414


ELBO: -738.8646240234375, log p(y): -1251.74169921875, dfs: tensor([10.3639], grad_fn=<SliceBackward>)d>)


ELBO: -41.507564544677734, log p(y): -56.031463623046875, dfs: tensor([11.3033, 11.3000], grad_fn=<SliceBackward>)


ELBO: -98.81013488769531, log p(y): -43.7799911499023446


ELBO: -40.30895233154297, log p(y): -49.33129119873047, dfs: tensor([10.4764], grad_fn=<SliceBackward>)))


...

ELBO: -41.34073257446289, log p(y): -36.4863700866699294


ELBO: -71.81057739257812, log p(y): -34.68627166748047, dfs: tensor([10.4843], grad_fn=<SliceBackward>)))


ELBO: -113.80889129638672, log p(y): -46.116477966308594, dfs: tensor([11.3950, 11.7898], grad_fn=<SliceBackward>)


ELBO: -44.562137603759766, log p(y): -42.638912200927734


KeyboardInterrupt: 

In [2]:
import pickle
#pickle.dump(data, open("8schools.pkl", "wb"))
data = pickle.load(open("8schools.pkl", "rb"))

In [3]:
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")

df = pd.DataFrame(data).melt(id_vars=['method','num_layers'])

In [6]:
df.groupby(['method', 'num_layers', 'variable']).describe()

value                                               \
                           count       mean        std         min        25%   
method num_layers variable                                                      
ADVI   1          elbo      88.0 -74.565299  76.488026 -482.983826 -59.313067   
                  log_p_y   94.0 -56.698815  55.716843 -459.081085 -52.720526   
       2          elbo      94.0 -80.470191  81.464868 -473.279572 -69.445307   
                  log_p_y   93.0 -56.898600  41.548223 -360.197235 -56.247681   
       3          elbo      90.0 -72.125958  68.872191 -438.706757 -62.673600   
                  log_p_y   95.0 -53.245201  34.382593 -280.727661 -52.752209   
       4          elbo      90.0 -62.267590  55.185163 -453.754242 -57.743567   
                  log_p_y   92.0 -56.440278  45.681035 -418.857361 -54.113357   
ATAF   1          elbo      90.0 -55.870983  30.015574 -192.163437 -55.038551   
                  log_p_y   95.0 -57.087304  53.109338 -464.434814 -52.803747   
       2          elbo      89.0 -65.382374  49.351925 -357.545807 -60.649368   
                  log_p_y   93.0 -51.076928  31.135507 -241.192719 -52.115978   
       3          elbo      90.0 -67.557421  61.966291 -440.284821 -59.311094   
                  log_p_y   93.0 -64.399045  79.888081 -475.118134 -50.133930   
       4          elbo      92.0 -58.628437  47.539362 -450.038544 -57.620953   
                  log_p_y   95.0 -51.010189  37.114505 -332.105988 -48.543411   
TAF    1          elbo      88.0 -64.637292  48.823274 -290.167297 -60.460025   
                  log_p_y   92.0 -52.508265  44.142359 -398.272003 -51.490613   
       2          elbo      92.0 -64.906408  57.260677 -366.458496 -58.218320   
                  log_p_y   90.0 -51.161705  33.910668 -314.408630 -50.262194   
       3          elbo      92.0 -59.810788  42.003284 -347.755310 -57.608432   
                  log_p_y   92.0 -56.414215  48.447030 -362.140717 -49.298287   
       4          elbo      92.0 -68.295279  58.502703 -362.432343 -61.794946   
                  log_p_y   91.0 -62.926512  70.271636 -401.731537 -50.621080   

                                                             
                                  50%        75%        max  
method num_layers variable                                   
ADVI   1          elbo     -49.750154 -44.317435 -31.664984  
                  log_p_y  -45.957985 -39.757118 -30.215361  
       2          elbo     -53.196482 -46.649135 -34.174564  
                  log_p_y  -45.585403 -39.969406 -30.553160  
       3          elbo     -49.540743 -43.286770 -33.777901  
                  log_p_y  -43.287773 -38.691462 -31.347588  
       4          elbo     -49.976957 -43.834058 -32.915718  
                  log_p_y  -45.277983 -40.028049 -31.524752  
ATAF   1          elbo     -47.500893 -42.728934 -32.650993  
                  log_p_y  -42.145542 -36.262796 -30.376169  
       2          elbo     -48.833736 -41.918022 -32.204105  
                  log_p_y  -43.686081 -36.146996 -30.389791  
       3          elbo     -49.264608 -42.136057 -33.461647  
                  log_p_y  -43.055935 -38.160767 -30.549675  
       4          elbo     -48.366121 -43.536874 -33.154083  
                  log_p_y  -42.948830 -37.902157 -31.561775  
TAF    1          elbo     -48.745274 -43.780972 -33.130344  
                  log_p_y  -42.186689 -37.599786 -30.970329  
       2          elbo     -48.634785 -44.100138 -32.953373  
                  log_p_y  -44.836164 -38.506952 -31.206520  
       3          elbo     -47.790285 -42.476170 -33.314434  
                  log_p_y  -42.900158 -39.962363 -31.279680  
       4          elbo     -48.894829 -43.262448 -32.999825  
                  log_p_y  -42.473045 -37.426815 -30.750813

In [21]:
gs = my_pdb.posterior("eight_schools-eight_schools_centered").reference_draws()

In [23]:
gs = my_pdb.posterior("eight_schools-eight_schools_centered").reference_draws()
gs_dict = {}
num_chains = len(gs)
num_samples = len(gs[0][next(iter(gs[0]))])

for i,chain in enumerate(gs):
    for var in chain:
        if '[' not in var:
            if var not in gs_dict:
                gs_dict[var] = np.zeros((num_chains, num_samples))
            gs_dict[var][i,:] = np.array(chain[var])
        else:
            name = var.split('[')[0]
            idx = int(var.split('[')[1].split(']')[0]) - 1
            if name not in gs_dict:
                var_size = len(list(filter(lambda x: x.startswith(name), chain)))
                gs_dict[name] = np.zeros((num_chains,num_samples,var_size))
            gs_dict[name][i,:,idx] = np.array(chain[var])

In [57]:
import torch.distributions as dist
def p_gs(theta, mu, tau):
    return (
        dist.Independent(dist.Normal(theta, sigma), 1).log_prob(y) +
        dist.Normal(mu.outer(torch.ones(8)), tau.outer(torch.ones(8))).log_prob(theta).sum(1)
    )
ps = p_gs(
    torch.tensor(gs_dict['theta']).flatten(end_dim=1),
    torch.tensor(gs_dict['mu']).flatten(end_dim=1),
    torch.tensor(gs_dict['tau']).flatten(end_dim=1),
)
print(ps.mean(), ps.std() / np.sqrt(ps.shape[0]))

tensor(-47.7822, dtype=torch.float64) tensor(0.0937, dtype=torch.float64)
